In [15]:
from utils.aqi_calculation import AqiCalculation
from utils.parsing_table import ParsingTable
from datetime import date
import pandas as pd
import numpy as np
import os
pd.options.mode.chained_assignment = None

path = '../resources/aqi_breakpoints.xlsx'
aqi = AqiCalculation(dict_df=pd.read_excel(path, sheet_name=['electrosmog_lf', 'wifi_level',
                                                                 'temperature', 'humidity', 'air_pressure',
                                                                 'ambient_light', 'tvoc',
                                                                 'co2', 'co2e', 'pm10', 'pm25', 'sound',
                                                                 'electrosmog_hf']))

input_path = '../resources/clean_file/'
dir_list = os.listdir(input_path)
meteo = pd.read_csv('../resources/meteo.csv').rename(columns={'DATA':'date','TMEDIA °C':'ext_temperature','UMIDITA %':'ext_humidity','VENTOMEDIA km/h':'ext_wind','PRESSIONESLM mb':'ext_pressure','FENOMENI':'ext_weather'})[['date','ext_temperature','ext_humidity','ext_wind','ext_pressure','ext_weather']]
meteo['ext_weather']= meteo['ext_weather'].fillna('sole')
meteo['date']=pd.to_datetime(meteo['date'],format="%d/%m/%Y")
pool = {}
pool_by_day={}
pt = ParsingTable()
for file in dir_list:
    if file.endswith('.csv'):
        temp: pd.DataFrame
        device_id:int
        naming = file.split('.')[0]
        print(naming)
        temp = pt.parse(input_path=input_path + file)
        temp['date'] = pd.to_datetime(temp['date'])
        pool[naming] = temp
        device_id = temp['device_id'].iloc[0]
        temp = temp.groupby(temp['date'].dt.date).agg({'iaq': 'mean', 'electrosmog_lf': 'mean', 'wifi_level': 'mean',
                                               'wifi_n': 'mean', 'temperature': 'mean', 'humidity': 'mean',
                                               'air_pressure': 'mean', 'ambient_light': 'mean', 'tvoc': 'mean',
                                               'co2': 'mean', 'co2e': 'mean', 'pm10': 'mean', 'pm25': 'mean',
                                               'sound': 'mean', 'electrosmog_hf': 'mean'}).reset_index()
        temp['device_id']=device_id
        temp['date'] = pd.to_datetime(temp['date'])
        temp = temp.merge(meteo, on='date')
        pool_by_day[naming] = temp


df_arpa = pd.read_csv('../resources/dati_nrt_informambiente_mv.csv')

print('end loading')

CAP153_1


C:\Users\lucab\Documents\Python\Assignement_RBS\src\utils\parsing_table.py:127: DtypeWarning: Columns (11,12,13,14,15,16,17,18,19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(str(path),


CAP153_2
CAP153_3


C:\Users\lucab\Documents\Python\Assignement_RBS\src\utils\parsing_table.py:127: DtypeWarning: Columns (13,14,15,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(str(path),


CAP186_1
CAP186_2


C:\Users\lucab\Documents\Python\Assignement_RBS\src\utils\parsing_table.py:127: DtypeWarning: Columns (13,14,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(str(path),


CAP186_3
end loading


In [22]:
clean_file_153 = pd.concat([pool['CAP153_1'],pool['CAP153_2'],pool['CAP153_3']])
clean_file_153.to_csv('../resources/concat_files/all_153.csv')

clean_file_186 = pd.concat([pool['CAP186_1'],pool['CAP186_2'],pool['CAP186_3']])
clean_file_186.to_csv('../resources/concat_files/all_186.csv')

clean_file_153_meteo =  pd.concat([pool_by_day['CAP153_1'],pool_by_day['CAP153_2'],pool_by_day['CAP153_3']])
clean_file_153_meteo.to_csv('../resources/concat_files/all_and_meteo_153.csv')

clean_file_186_meteo = pd.concat([pool_by_day['CAP186_1'],pool_by_day['CAP186_2'],pool_by_day['CAP186_3']])
clean_file_186_meteo.to_csv('../resources/concat_files/all_and_meteo_186.csv')





In [8]:
test = pool['CAP186_2'].copy()

test = test[ (test['battery'] != 'good') & (test['battery'] != 'low') ]

test

,device_id,iaq,electrosmog_lf,wifi_level,wifi_n,temperature,humidity,air_pressure,ambient_light,tvoc,...,pm25,electrosmog_hf,sound,battery,main_power,air,comfort,electrosmog,date,co2


In [ ]:
device_number: pd.DataFrame = pd.DataFrame()
device_number = device_number.assign(device_id=pool_33_3['device_id'],
                                     device_n=pool_33_3.groupby('device_id')['device_id'].transform(
                                         'count')).drop_duplicates()
# pool_33_3['dev_n'] = pool_33_3.groupby('device_id')['device_id'].transform('count')
device_number

In [ ]:
# oaq --> iaq (con stesse variabili conosciute) ==> troviamo efficienza filtraggio interno